In [ ]:
# Load dependencies

import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from scipy.stats.stats import pearsonr
from matplotlib import pyplot as plt

In [ ]:
# Read in the structured correlation data
# Update filename based on epoch duration

corr_df = pd.read_csv("preprocessed_interpolate_bandpass_.1_20/merged_corr_df_1s_epoch.csv")
len(corr_df)

In [ ]:
# This filters out extreme values that are > +/-.4
# This is a brute-force way of excluding outliers
filter_index = (corr_df[["TP10", "AF7", "AF8", "TP9"]] > 0.4).any(1)  | (corr_df[["TP10", "AF7", "AF8", "TP9"]] < -0.4).any(1) | (corr_df["Signal Drop"] == 1)

# Print how many are filtered out
filter_index.sum()

In [ ]:
# Filter out participants from the filter index

corr_df = corr_df[~filter_index]

In [ ]:
# Filter our participants who self-report that they possibly knew eachother

# List of subjects who possibly knew each other
subjects_to_filter = [27, 38, 40, 43, 54, 56, 66, 87, 103, 112, 160, 20, 36, 62, 71, 102, 111, 133, 143, 180, 15, 19, 53, 64, 116]

# Create a filter mask for the DataFrame
know_filter_mask = corr_df["subj"].isin(subjects_to_filter)

# Sum the mask to count how many entries are True (i.e., how many rows have subjects that need to be filtered)
know_filter_mask.sum()


In [ ]:
# IMPORTANT!!!
# THIS IS FOR SENSITIVITY ANALYSIS
# ONLY UNCOMMENT IF YOU WANT TO EXCLUDE PARTICIPANTS WHO SELF-REPORTED THAT THEY MIGHT KNOW EACHOTHER
# IF YOU RUN THIS, DON'T RUN THE PLOTTING CODE, UNLESS YOU WANT TO OVERWRITE YOUR PLOTS ON THE "COMPLETE" DATAST

# Filter out participants from the know filter mask

#corr_df = corr_df[~know_filter_mask]

In [ ]:
# Rename the time variable for easier modeling

corr_df = corr_df.rename(columns={'Time (sec)': 'Time'})

In [ ]:
#corr_df_clean_electrodes.to_csv('corr_df_clean_electrodes.csv', index=False)

In [ ]:
# create a dataframe that does listwise deletion for missing electrode or behavioral data
corr_df_clean_electrodes = corr_df.dropna(subset=["TP9", "AF7", "AF8", "TP10", "Performance", "Time"])

# create a dataframe that does listwise deletion for missing electrode or performance behavioral data
corr_df_clean_electrodes_performance = corr_df.dropna(subset=["TP9", "AF7", "AF8", "TP10", "Performance"])

# create a dataframe that does listwise deletion for missing electrode or performance time data
corr_df_clean_electrodes_time = corr_df.dropna(subset=["TP9", "AF7", "AF8", "TP10", "Time"])

# create a dataframe that does listwise deletion for missing electrode or self-report
corr_df_clean_trust = corr_df.dropna(subset=["TP9", "AF7", "AF8", "TP10", "affective_trust", "cognitive_trust", "affecttrust_corr", "cognitivetrust_corr"])

In [ ]:
# Get some data on the impact of listwise deletion

print(f"The corr_df dataframe is {len(corr_df)} rows long")
print(f"The corr_df_clean_electrodes dataframe is {len(corr_df_clean_electrodes)} rows long")
print(f"The corr_df_clean_electrodes_performance dataframe is {len(corr_df_clean_electrodes_performance)} rows long")
print(f"The corr_df_clean_electrodes_time is {len(corr_df_clean_electrodes_time)} rows long")
print(f"The corr_df_clean_trust is {len(corr_df_clean_trust)} rows long")

print(f"The corr_df dataframe has {corr_df['subj'].nunique()} unique participants")
print(f"The corr_df_clean_electrodes has {corr_df_clean_electrodes['subj'].nunique()} unique participants")
print(f"The corr_df_clean_electrodes_performance has {corr_df_clean_electrodes_performance['subj'].nunique()} unique participants")
print(f"The corr_df_clean_electrodes_time has {corr_df_clean_electrodes_time['subj'].nunique()} unique participants")
print(f"The corr_df_clean_trust  has {corr_df_clean_trust ['subj'].nunique()} unique participants")


In [ ]:
# Get some insights into the data
sns.pairplot(corr_df[["TP9", "AF7", "AF8", "TP10", "Performance", "Time", "affective_trust", "cognitive_trust", "affecttrust_corr", "cognitivetrust_corr" ]])
plt.show()

In [ ]:
# Test if performance and time vary as a function of session

# Fit a mixed model
# This formula specifies a model with:
    # time/performamce as the DV
    # ses as a fixed effect
    # subject as a random intercepts

model_performance_ses = smf.mixedlm("Performance ~ ses", corr_df_clean_electrodes, groups=corr_df_clean_electrodes["subj"])
model_time_ses = smf.mixedlm("Time ~ ses", corr_df_clean_electrodes, groups=corr_df_clean_electrodes["subj"])

# Fit the model
result_performance_ses = model_performance_ses.fit()
result_time_ses = model_time_ses.fit()

# Print the summary of the model fit
print(result_performance_ses.summary())
print(result_time_ses.summary())

In [ ]:
# Plot boxplot of performance by session
plt.figure(figsize=(8.5, 4.5))

# Define color palette for ses levels
color_palette = {1: '#4d004b', 2: '#88419d', 3: '#8c96c6'}

# Setting the color palette in seaborn
sns.set_palette(list(color_palette.values()))


# Plot boxplots for cognitivetrust_corr
plt.subplot(1, 2, 1)
sns.boxplot(x='ses', y='Performance', data=corr_df, palette=color_palette)
plt.title('Number of Correctly Matched Tangrams by Session')
plt.xlabel('Session')  # Set horizontal axis label
plt.ylabel('Performance')  # Set vertical axis label

# Plot boxplots for affecttrust_corr
plt.subplot(1, 2, 2)
sns.boxplot(x='ses', y='Time', data=corr_df_clean_trust, palette=color_palette)
plt.title('Time to Complete a Tangram Round by Session')
plt.xlabel('Session')  # Set horizontal axis label
plt.ylabel('Time')  # Set vertical axis label

# Adjust layout
plt.tight_layout()

# Save the figure
plt.savefig('plots/behavior_by_session_boxplot.png', dpi=300)

# Show the plot
plt.show()


In [ ]:
# Calculate the cell means for 'Performance' for each 'ses'
# uses stats.ttest_rel which is a paired-samples t-test that deals with the repeated measure

# Pivot the table to have ses levels as columns and each row representing a subject
pivot_df = corr_df_clean_electrodes.pivot_table(index='subj', columns='ses', values='Performance')

# Print cell means for each level of 'ses' within subjects
print("Cell means for each ses level within subjects:")
print(pivot_df.mean())  # This will calculate the mean for each session across subjects
print("\nStandard deviations for each ses level within subjects:")
print(pivot_df.std())  # Calculates the standard deviation for each session across subjects

# Code for conducting pairwise comparisons
ses_levels = pivot_df.columns
results = []
for i in range(len(ses_levels)):
    for j in range(i + 1, len(ses_levels)):
        # Drop NaN values to ensure both sessions have data points for all subjects
        valid_data = pivot_df.dropna(subset=[ses_levels[i], ses_levels[j]])
        if not valid_data.empty:
            t_stat, p_val = stats.ttest_rel(valid_data[ses_levels[i]], valid_data[ses_levels[j]])
            dof = len(valid_data) - 1  # Degrees of freedom for the paired t-test
            results.append({
                'ses1': ses_levels[i],
                'ses2': ses_levels[j],
                't_stat': t_stat,
                'p_val': p_val,
                'dof': dof
            })

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)
print("Pairwise T-test results:")
print(results_df)


In [ ]:
# Calculate the cell means for 'Time' for each 'ses'
# uses stats.ttest_rel which is a paired-samples t-test that deals with the repeated measure

# Pivot the table to have ses levels as columns and each row representing a subject
pivot_df = corr_df_clean_electrodes.pivot_table(index='subj', columns='ses', values='Time')

# Print cell means for each level of 'ses' within subjects
print("Cell means for each ses level within subjects:")
print(pivot_df.mean())  # This will calculate the mean for each session across subjects
print("\nStandard deviations for each ses level within subjects:")
print(pivot_df.std())  # Calculates the standard deviation for each session across subjects

# Code for conducting pairwise comparisons
ses_levels = pivot_df.columns
results = []
for i in range(len(ses_levels)):
    for j in range(i + 1, len(ses_levels)):
        # Drop NaN values to ensure both sessions have data points for all subjects
        valid_data = pivot_df.dropna(subset=[ses_levels[i], ses_levels[j]])
        if not valid_data.empty:
            t_stat, p_val = stats.ttest_rel(valid_data[ses_levels[i]], valid_data[ses_levels[j]])
            dof = len(valid_data) - 1  # Degrees of freedom for the paired t-test
            results.append({
                'ses1': ses_levels[i],
                'ses2': ses_levels[j],
                't_stat': t_stat,
                'p_val': p_val,
                'dof': dof
            })

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)
print("Pairwise T-test results:")
print(results_df)


In [ ]:
# Test if ISCs in a given electrode increase as a function of session
# This would test if participant ISCs increase over time

# Fit a mixed model
# This formula specifies a model with:
    # ISC for a given electrode as the DV
    # ses as a fixed effect
    # subject as a random intercepts

model_af7_isc_ses = smf.mixedlm("AF7 ~ ses", corr_df_clean_electrodes, groups=corr_df_clean_electrodes["subj"])
model_af8_isc_ses = smf.mixedlm("AF8 ~ ses", corr_df_clean_electrodes, groups=corr_df_clean_electrodes["subj"])
model_tp9_isc_ses = smf.mixedlm("TP9 ~ ses", corr_df_clean_electrodes, groups=corr_df_clean_electrodes["subj"])
model_tp10_isc_ses = smf.mixedlm("TP10 ~ ses", corr_df_clean_electrodes, groups=corr_df_clean_electrodes["subj"])

# Fit the model
result_af7_isc_ses = model_af7_isc_ses.fit()
result_af8_isc_ses = model_af8_isc_ses.fit()
result_tp9_isc_ses = model_tp9_isc_ses.fit()
result_tp10_isc_ses = model_tp10_isc_ses.fit()

# Print the summary of the model fit
print(result_af7_isc_ses.summary())
print(result_af8_isc_ses.summary())
print(result_tp9_isc_ses.summary())
print(result_tp10_isc_ses.summary())

In [ ]:
# Plot boxplot of electrode ISC by session
fig, axs = plt.subplots(2, 2, figsize=(8.5, 8.5), dpi=300)  # Set figure size to 8.5x8.5 inches

channels = ["AF7", "AF8", "TP9", "TP10"]
# Define custom color palette, from color brewer
color_palette = {1: '#4d004b', 2: '#88419d', 3: '#8c96c6'}

for i, channel in enumerate(channels):
    sns.boxplot(x="ses", y=channel, data=corr_df_clean_electrodes, ax=axs[i//2, i%2],
                palette=color_palette)  # Use the custom palette for box plots
    axs[i//2, i%2].set_title(f"{channel} ISC by Session")  # Set title for each subplot
    axs[i//2, i%2].set_ylabel(channel)  # Explicitly set y-axis label to the channel name
    axs[i//2, i%2].set_ylim(-0.4, 0.4)  # Set the y-axis range to be from -0.3 to 0.3
    axs[i//2, i%2].set_xlabel("Session")  # Set x-axis label to "Session"

# Ensure the background of the figure is white
fig.patch.set_facecolor('white')

plt.tight_layout(pad=2.0)  # Increase padding between plots for clarity
plt.savefig("plots/isc_by_session_boxplot.png", bbox_inches='tight', facecolor=fig.get_facecolor())  # Save the figure as a PNG file
plt.show()


In [ ]:
# Calculate the cell means for 'AF7' for each 'ses'
# uses stats.ttest_rel which is a paired-samples t-test that deals with the repeated measure

# Pivot the table to have ses levels as columns and each row representing a subject
pivot_df = corr_df_clean_electrodes.pivot_table(index='subj', columns='ses', values='AF7')

# Print cell means for each level of 'ses' within subjects
print("Cell means for each ses level within subjects:")
print(pivot_df.mean())  # This will calculate the mean for each session across subjects
print("\nStandard deviations for each ses level within subjects:")
print(pivot_df.std())  # Calculates the standard deviation for each session across subjects

# Code for conducting pairwise comparisons
ses_levels = pivot_df.columns
results = []
for i in range(len(ses_levels)):
    for j in range(i + 1, len(ses_levels)):
        # Drop NaN values to ensure both sessions have data points for all subjects
        valid_data = pivot_df.dropna(subset=[ses_levels[i], ses_levels[j]])
        if not valid_data.empty:
            t_stat, p_val = stats.ttest_rel(valid_data[ses_levels[i]], valid_data[ses_levels[j]])
            dof = len(valid_data) - 1  # Degrees of freedom for the paired t-test
            results.append({
                'ses1': ses_levels[i],
                'ses2': ses_levels[j],
                't_stat': t_stat,
                'p_val': p_val,
                'dof': dof
            })

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)
print("Pairwise T-test results:")
print(results_df)


In [ ]:
# Calculate the cell means for 'AF8' for each 'ses'
# uses stats.ttest_rel which is a paired-samples t-test that deals with the repeated measure

# Pivot the table to have ses levels as columns and each row representing a subject
pivot_df = corr_df_clean_electrodes.pivot_table(index='subj', columns='ses', values='AF8')

# Print cell means for each level of 'ses' within subjects
print("Cell means for each ses level within subjects:")
print(pivot_df.mean())  # This will calculate the mean for each session across subjects
print("\nStandard deviations for each ses level within subjects:")
print(pivot_df.std())  # Calculates the standard deviation for each session across subjects

# Code for conducting pairwise comparisons
ses_levels = pivot_df.columns
results = []
for i in range(len(ses_levels)):
    for j in range(i + 1, len(ses_levels)):
        # Drop NaN values to ensure both sessions have data points for all subjects
        valid_data = pivot_df.dropna(subset=[ses_levels[i], ses_levels[j]])
        if not valid_data.empty:
            t_stat, p_val = stats.ttest_rel(valid_data[ses_levels[i]], valid_data[ses_levels[j]])
            dof = len(valid_data) - 1  # Degrees of freedom for the paired t-test
            results.append({
                'ses1': ses_levels[i],
                'ses2': ses_levels[j],
                't_stat': t_stat,
                'p_val': p_val,
                'dof': dof
            })

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)
print("Pairwise T-test results:")
print(results_df)


In [ ]:
# Calculate the cell means for 'TP9' for each 'ses'
# uses stats.ttest_rel which is a paired-samples t-test that deals with the repeated measure

# Pivot the table to have ses levels as columns and each row representing a subject
pivot_df = corr_df_clean_electrodes.pivot_table(index='subj', columns='ses', values='TP9')

# Print cell means for each level of 'ses' within subjects
print("Cell means for each ses level within subjects:")
print(pivot_df.mean())  # This will calculate the mean for each session across subjects
print("\nStandard deviations for each ses level within subjects:")
print(pivot_df.std())  # Calculates the standard deviation for each session across subjects

# Code for conducting pairwise comparisons
ses_levels = pivot_df.columns
results = []
for i in range(len(ses_levels)):
    for j in range(i + 1, len(ses_levels)):
        # Drop NaN values to ensure both sessions have data points for all subjects
        valid_data = pivot_df.dropna(subset=[ses_levels[i], ses_levels[j]])
        if not valid_data.empty:
            t_stat, p_val = stats.ttest_rel(valid_data[ses_levels[i]], valid_data[ses_levels[j]])
            dof = len(valid_data) - 1  # Degrees of freedom for the paired t-test
            results.append({
                'ses1': ses_levels[i],
                'ses2': ses_levels[j],
                't_stat': t_stat,
                'p_val': p_val,
                'dof': dof
            })

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)
print("Pairwise T-test results:")
print(results_df)


In [ ]:
# Calculate the cell means for 'TP10' for each 'ses'
# uses stats.ttest_rel which is a paired-samples t-test that deals with the repeated measure

# Pivot the table to have ses levels as columns and each row representing a subject
pivot_df = corr_df_clean_electrodes.pivot_table(index='subj', columns='ses', values='TP10')

# Print cell means for each level of 'ses' within subjects
print("Cell means for each ses level within subjects:")
print(pivot_df.mean())  # This will calculate the mean for each session across subjects
print("\nStandard deviations for each ses level within subjects:")
print(pivot_df.std())  # Calculates the standard deviation for each session across subjects

# Code for conducting pairwise comparisons
ses_levels = pivot_df.columns
results = []
for i in range(len(ses_levels)):
    for j in range(i + 1, len(ses_levels)):
        # Drop NaN values to ensure both sessions have data points for all subjects
        valid_data = pivot_df.dropna(subset=[ses_levels[i], ses_levels[j]])
        if not valid_data.empty:
            t_stat, p_val = stats.ttest_rel(valid_data[ses_levels[i]], valid_data[ses_levels[j]])
            dof = len(valid_data) - 1  # Degrees of freedom for the paired t-test
            results.append({
                'ses1': ses_levels[i],
                'ses2': ses_levels[j],
                't_stat': t_stat,
                'p_val': p_val,
                'dof': dof
            })

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)
print("Pairwise T-test results:")
print(results_df)


In [ ]:
# Test if Average Cognitive and Affective Trust increases as a function of session
# This would test if participant average cognitive and trust increase over time

# Fit a mixed model
# This formula specifies a model with:
    # average trust scale for a trust scale
    # ses as a fixed effect
    # subject as a random intercepts

model_average_cognitive_trust_ses = smf.mixedlm("cognitive_trust ~ ses", corr_df_clean_trust, groups=corr_df_clean_trust["subj"])
model_average_affective_trust_ses = smf.mixedlm("affective_trust ~ ses", corr_df_clean_trust, groups=corr_df_clean_trust["subj"])


# Fit the model
result_average_cognitive_trust_ses = model_average_cognitive_trust_ses.fit()
result_average_affective_trust_ses = model_average_affective_trust_ses.fit()


# Print the summary of the model fit
print(result_average_cognitive_trust_ses.summary())
print(result_average_affective_trust_ses.summary())


In [ ]:
# Plot boxplot of trust by session
plt.figure(figsize=(8.5, 4.5))

# Define color palette for ses levels
color_palette = {1: '#4d004b', 2: '#88419d', 3: '#8c96c6'}

# Setting the color palette in seaborn
sns.set_palette(list(color_palette.values()))


# Plot boxplots for cognitivetrust_corr
plt.subplot(1, 2, 1)
sns.boxplot(x='ses', y='cognitive_trust', data=corr_df, palette=color_palette)
plt.title('Average Cognitive Trust by Session')
plt.xlabel('Session')  # Set horizontal axis label
plt.ylabel('Pairwise Average')  # Set vertical axis label

# Plot boxplots for affecttrust_corr
plt.subplot(1, 2, 2)
sns.boxplot(x='ses', y='affective_trust', data=corr_df_clean_trust, palette=color_palette)
plt.title('Average Affective Trust by Session')
plt.xlabel('Session')  # Set horizontal axis label
plt.ylabel('Pairwise Average')  # Set vertical axis label

# Adjust layout
plt.tight_layout()

# Save the figure
plt.savefig('plots/average_trust_by_session_boxplot.png', dpi=300)

# Show the plot
plt.show()


In [ ]:
# Calculate the cell means for 'cognitive_trust' for each 'ses'
# uses stats.ttest_rel which is a paired-samples t-test that deals with the repeated measure

# Pivot the table to have ses levels as columns and each row representing a subject
pivot_df = corr_df_clean_trust.pivot_table(index='subj', columns='ses', values='cognitive_trust')

# Print cell means for each level of 'ses' within subjects
print("Cell means for each ses level within subjects:")
print(pivot_df.mean())  # This will calculate the mean for each session across subjects
print("\nStandard deviations for each ses level within subjects:")
print(pivot_df.std())  # Calculates the standard deviation for each session across subjects

# Code for conducting pairwise comparisons
ses_levels = pivot_df.columns
results = []
for i in range(len(ses_levels)):
    for j in range(i + 1, len(ses_levels)):
        # Drop NaN values to ensure both sessions have data points for all subjects
        valid_data = pivot_df.dropna(subset=[ses_levels[i], ses_levels[j]])
        if not valid_data.empty:
            t_stat, p_val = stats.ttest_rel(valid_data[ses_levels[i]], valid_data[ses_levels[j]])
            dof = len(valid_data) - 1  # Degrees of freedom for the paired t-test
            results.append({
                'ses1': ses_levels[i],
                'ses2': ses_levels[j],
                't_stat': t_stat,
                'p_val': p_val,
                'dof': dof
            })

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)
print("Pairwise T-test results:")
print(results_df)


In [ ]:
# Calculate the cell means for 'affective_trust' for each 'ses'
# uses stats.ttest_rel which is a paired-samples t-test that deals with the repeated measure

# Pivot the table to have ses levels as columns and each row representing a subject
pivot_df = corr_df_clean_trust.pivot_table(index='subj', columns='ses', values='affective_trust')

# Print cell means for each level of 'ses' within subjects
print("Cell means for each ses level within subjects:")
print(pivot_df.mean())  # This will calculate the mean for each session across subjects
print("\nStandard deviations for each ses level within subjects:")
print(pivot_df.std())  # Calculates the standard deviation for each session across subjects

# Code for conducting pairwise comparisons
ses_levels = pivot_df.columns
results = []
for i in range(len(ses_levels)):
    for j in range(i + 1, len(ses_levels)):
        # Drop NaN values to ensure both sessions have data points for all subjects
        valid_data = pivot_df.dropna(subset=[ses_levels[i], ses_levels[j]])
        if not valid_data.empty:
            t_stat, p_val = stats.ttest_rel(valid_data[ses_levels[i]], valid_data[ses_levels[j]])
            dof = len(valid_data) - 1  # Degrees of freedom for the paired t-test
            results.append({
                'ses1': ses_levels[i],
                'ses2': ses_levels[j],
                't_stat': t_stat,
                'p_val': p_val,
                'dof': dof
            })

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)
print("Pairwise T-test results:")
print(results_df)


In [ ]:
# NOTE: AFFECTIVE TRUST MODEL IS SINGULAR AND FAILS TO CONVERGE
# you fit a simple OSL in the next cell

# Test if Pairweise Cognitive and Affective Trust correlation increases as a function of session
# This would test if participant pairwise cognitive and trust correlations increase over time

# Fit a mixed model
# This formula specifies a model with:
    # pairwise trust correlation for a trust scale
    # ses as a fixed effect
    # subject as a random intercepts

model_correl_cognitive_trust_ses = smf.mixedlm("cognitivetrust_corr ~ ses", corr_df_clean_trust, groups=corr_df_clean_trust["subj"])
model_correl_affective_trust_ses = smf.mixedlm("affecttrust_corr ~ ses", corr_df_clean_trust, groups=corr_df_clean_trust["subj"])


# Fit the model
result_correl_cognitive_trust_ses = model_correl_cognitive_trust_ses.fit()
result_correl_affective_trust_ses = model_correl_affective_trust_ses.fit()


# Print the summary of the model fit
print(result_correl_cognitive_trust_ses.summary())
print(result_correl_affective_trust_ses.summary())


In [ ]:
# Simple OLS since the more complicated model didn't work

model_correl_affective_trust_ses_ols = smf.ols("affecttrust_corr ~ ses", data=corr_df_clean_trust)
result_correl_affective_trust_ses_ols = model_correl_affective_trust_ses_ols.fit()
print(result_correl_affective_trust_ses_ols.summary())

In [ ]:
# Plot boxplot of correlation of trust by session
plt.figure(figsize=(8.5, 4.5))

# Define color palette for ses levels
color_palette = {1: '#4d004b', 2: '#88419d', 3: '#8c96c6'}

# Setting the color palette in seaborn
sns.set_palette(list(color_palette.values()))

# Plot boxplots for cognitivetrust_corr
plt.subplot(1, 2, 1)
sns.boxplot(x='ses', y='cognitivetrust_corr', data=corr_df, palette=color_palette)
plt.title('Pairwise Cognitive Trust Correlation by Session')
plt.xlabel('Session')  # Set horizontal axis label
plt.ylabel('Pairwise Correlation')  # Set vertical axis label

# Plot boxplots for affecttrust_corr
plt.subplot(1, 2, 2)
sns.boxplot(x='ses', y='affecttrust_corr', data=corr_df_clean_trust, palette=color_palette)
plt.title('Pairwise Affective Trust Correlation by Session')
plt.xlabel('Session')  # Set horizontal axis label
plt.ylabel('Pairwise Correlation')  # Set vertical axis label

# Adjust layout
plt.tight_layout()

# Save the figure
plt.savefig('plots/correl_trust_by_session_boxplot.png', dpi=300)

# Show the plot
plt.show()


In [ ]:
# Calculate the cell means for 'cognitivetrust_corr' for each 'ses'
# uses stats.ttest_rel which is a paired-samples t-test that deals with the repeated measure

# Pivot the table to have ses levels as columns and each row representing a subject
pivot_df = corr_df_clean_trust.pivot_table(index='subj', columns='ses', values='cognitivetrust_corr')

# Print cell means for each level of 'ses' within subjects
print("Cell means for each ses level within subjects:")
print(pivot_df.mean())  # This will calculate the mean for each session across subjects
print("\nStandard deviations for each ses level within subjects:")
print(pivot_df.std())  # Calculates the standard deviation for each session across subjects

# Code for conducting pairwise comparisons
ses_levels = pivot_df.columns
results = []
for i in range(len(ses_levels)):
    for j in range(i + 1, len(ses_levels)):
        # Drop NaN values to ensure both sessions have data points for all subjects
        valid_data = pivot_df.dropna(subset=[ses_levels[i], ses_levels[j]])
        if not valid_data.empty:
            t_stat, p_val = stats.ttest_rel(valid_data[ses_levels[i]], valid_data[ses_levels[j]])
            dof = len(valid_data) - 1  # Degrees of freedom for the paired t-test
            results.append({
                'ses1': ses_levels[i],
                'ses2': ses_levels[j],
                't_stat': t_stat,
                'p_val': p_val,
                'dof': dof
            })

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)
print("Pairwise T-test results:")
print(results_df)


In [ ]:
# Calculate the cell means for 'affecttrust_corr' for each 'ses'
# uses stats.ttest_rel which is a paired-samples t-test that deals with the repeated measure

# Pivot the table to have ses levels as columns and each row representing a subject
pivot_df = corr_df_clean_trust.pivot_table(index='subj', columns='ses', values='affecttrust_corr')

# Print cell means for each level of 'ses' within subjects
print("Cell means for each ses level within subjects:")
print(pivot_df.mean())  # This will calculate the mean for each session across subjects
print("\nStandard deviations for each ses level within subjects:")
print(pivot_df.std())  # Calculates the standard deviation for each session across subjects

# Code for conducting pairwise comparisons
ses_levels = pivot_df.columns
results = []
for i in range(len(ses_levels)):
    for j in range(i + 1, len(ses_levels)):
        # Drop NaN values to ensure both sessions have data points for all subjects
        valid_data = pivot_df.dropna(subset=[ses_levels[i], ses_levels[j]])
        if not valid_data.empty:
            t_stat, p_val = stats.ttest_rel(valid_data[ses_levels[i]], valid_data[ses_levels[j]])
            dof = len(valid_data) - 1  # Degrees of freedom for the paired t-test
            results.append({
                'ses1': ses_levels[i],
                'ses2': ses_levels[j],
                't_stat': t_stat,
                'p_val': p_val,
                'dof': dof
            })

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)
print("Pairwise T-test results:")
print(results_df)


In [ ]:
# Fit a complex mixed model that deals with sessions nested within sujects
# This model:
    # "Performance ~ Electrode" defines "Performance" as the DV and"AF7" as an independent variable (fixed effect)
    # groups=corr_df["subj"] specifies that the random intercepts should vary by "subj" to account for the non-independence of observations within subjects
    #re_formula="1" indicates that there is a random intercept for each subject
    # vc_formula={"ses": "0 + C(ses)"} specifies a variance component for "ses", treating sessions as a categorical variable (with C(ses)) and allowing for varying intercepts by "ses" without a slope
# This may not converge, so we fit simpler models below

# Define the models
model_af7 = smf.mixedlm("Performance ~ AF7", data=corr_df_clean_electrodes_performance, groups=corr_df_clean_electrodes_performance["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_af8 = smf.mixedlm("Performance ~ AF8", data=corr_df_clean_electrodes_performance, groups=corr_df_clean_electrodes_performance["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp9 = smf.mixedlm("Performance ~ TP9", data=corr_df_clean_electrodes_performance, groups=corr_df_clean_electrodes_performance["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp10 = smf.mixedlm("Performance ~ TP10", data=corr_df_clean_electrodes_performance, groups=corr_df_clean_electrodes_performance["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

# Fit the model
results_af7 = model_af7.fit()
results_af8 = model_af8.fit()
results_tp9 = model_tp9.fit()
results_tp10 = model_tp10.fit()

# Print the summary of the model fit
print(results_af7.summary())
print(results_af8.summary())
print(results_tp9.summary())
print(results_tp10.summary())

In [ ]:
# Plot the mixed model of electrode on performance

# Prepare the grid for plotting
fig, axs = plt.subplots(2, 2, figsize=(8.5, 8.5), dpi=300)  # Adjust figsize to 8.5 x 8.5 inches

electrodes = ['AF7', 'AF8', 'TP9', 'TP10']

for i, electrode in enumerate(electrodes):
    # Fit the model for the current electrode
    model = smf.mixedlm(f"Performance ~ {electrode}", data=corr_df_clean_electrodes_performance, groups=corr_df_clean_electrodes_performance["subj"], re_formula="1",
                        vc_formula={"ses": "0 + C(ses)"}).fit()
    
    # Get predicted values for a range of electrode values
    electrode_values = np.linspace(corr_df_clean_electrodes_performance[electrode].min(), corr_df_clean_electrodes_performance[electrode].max(), 100)
    predicted_performance = model.params['Intercept'] + model.params[electrode] * electrode_values
    
    # Determine the subplot to use
    ax = axs[i // 2, i % 2]
    
    # Plot the observed data using the specified color
    sns.scatterplot(x=electrode, y='Performance', data=corr_df_clean_electrodes_performance, color='#8c96c6', alpha=0.6, ax=ax)
    
    # Plot the fitted line using the specified color
    ax.plot(electrode_values, predicted_performance, color='#4d004b', label='Fitted Line')
    
    # Set titles and labels
    ax.set_xlabel(electrode)
    ax.set_ylabel('Performance')
    ax.set_title(f'Mixed Model Fit: Performance ~ {electrode}')
    ax.legend()

# Ensure the background of the figure is white
fig.patch.set_facecolor('white')

plt.tight_layout()
plt.savefig('plots/model_fits_performance.png', format='png', dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')  # Save the figure
plt.show()


In [ ]:
# Now fit the same model for time

# Fit a complex mixed model that deals with sessions nested within sujects
# This model:
    # "Performance ~ Electrode" defines "Performance" as the DV and"AF7" as an independent variable (fixed effect)
    # groups=corr_df["subj"] specifies that the random intercepts should vary by "subj" to account for the non-independence of observations within subjects
    #re_formula="1" indicates that there is a random intercept for each subject
    # vc_formula={"ses": "0 + C(ses)"} specifies a variance component for "ses", treating sessions as a categorical variable (with C(ses)) and allowing for varying intercepts by "ses" without a slope
# This may not converge, so we fit simpler models below

# Define the models
model_af7 = smf.mixedlm("Time ~ AF7", data=corr_df_clean_electrodes_time, groups=corr_df_clean_electrodes_time["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_af8 = smf.mixedlm("Time ~ AF8", data=corr_df_clean_electrodes_time, groups=corr_df_clean_electrodes_time["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp9 = smf.mixedlm("Time ~ TP9", data=corr_df_clean_electrodes_time, groups=corr_df_clean_electrodes_time["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp10 = smf.mixedlm("Time ~ TP10", data=corr_df_clean_electrodes_time, groups=corr_df_clean_electrodes_time["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

# Fit the model
results_af7 = model_af7.fit()
results_af8 = model_af8.fit()
results_tp9 = model_tp9.fit()
results_tp10 = model_tp10.fit()

# Print the summary of the model fit
print(results_af7.summary())
print(results_af8.summary())
print(results_tp9.summary())
print(results_tp10.summary())

In [ ]:
# Plot the mixed model of electrode on time

# Prepare the grid for plotting
fig, axs = plt.subplots(2, 2, figsize=(8.5, 8.5), dpi=300)  # Adjust figsize to 8.5 x 8.5 inches

electrodes = ['AF7', 'AF8', 'TP9', 'TP10']

for i, electrode in enumerate(electrodes):
    # Fit the model for the current electrode
    model = smf.mixedlm(f"Time ~ {electrode}", data=corr_df_clean_electrodes_time, groups=corr_df_clean_electrodes_time["subj"], re_formula="1",
                        vc_formula={"ses": "0 + C(ses)"}).fit()
    
    # Get predicted values for a range of electrode values
    electrode_values = np.linspace(corr_df_clean_electrodes_time[electrode].min(), corr_df_clean_electrodes_time[electrode].max(), 100)
    predicted_performance = model.params['Intercept'] + model.params[electrode] * electrode_values
    
    # Determine the subplot to use
    ax = axs[i // 2, i % 2]
    
    # Plot the observed data using the specified color
    sns.scatterplot(x=electrode, y='Time', data=corr_df_clean_electrodes_time, color='#8c96c6', alpha=0.6, ax=ax)
    
    # Plot the fitted line using the specified color
    ax.plot(electrode_values, predicted_performance, color='#4d004b', label='Fitted Line')
    
    # Set titles and labels
    ax.set_xlabel(electrode)
    ax.set_ylabel('Time')
    ax.set_title(f'Mixed Model Fit: Time ~ {electrode}')
    ax.legend()

# Ensure the background of the figure is white
fig.patch.set_facecolor('white')

plt.tight_layout()
plt.savefig('plots/model_fits_time.png', format='png', dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')  # Save the figure
plt.show()


In [ ]:
# Now fit the same model for average cognitive trust

# Fit a complex mixed model that deals with sessions nested within sujects
# This model:
    # "Cognitive Trust ~ Electrode" defines "Cognitive Trust" as the DV and"AF7" as an independent variable (fixed effect)
    # groups=corr_df["subj"] specifies that the random intercepts should vary by "subj" to account for the non-independence of observations within subjects
    #re_formula="1" indicates that there is a random intercept for each subject
    # vc_formula={"ses": "0 + C(ses)"} specifies a variance component for "ses", treating sessions as a categorical variable (with C(ses)) and allowing for varying intercepts by "ses" without a slope
# This may not converge, so we fit simpler models below

# Define the models
model_af7 = smf.mixedlm("cognitive_trust ~ AF7", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_af8 = smf.mixedlm("cognitive_trust ~ AF8", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp9 = smf.mixedlm("cognitive_trust ~ TP9", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp10 = smf.mixedlm("cognitive_trust ~ TP10", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

# Fit the model
results_af7 = model_af7.fit()
results_af8 = model_af8.fit()
results_tp9 = model_tp9.fit()
results_tp10 = model_tp10.fit()

# Print the summary of the model fit
print(results_af7.summary())
print(results_af8.summary())
print(results_tp9.summary())
print(results_tp10.summary())

In [ ]:
# Plot the mixed model of electrode on average cognitive_trust

# Prepare the grid for plotting
fig, axs = plt.subplots(2, 2, figsize=(8.5, 8.5), dpi=300)  # Adjust figsize to 8.5 x 8.5 inches

electrodes = ['AF7', 'AF8', 'TP9', 'TP10']

for i, electrode in enumerate(electrodes):
    # Fit the model for the current electrode
    model = smf.mixedlm(f"cognitive_trust ~ {electrode}", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"], re_formula="1",
                        vc_formula={"ses": "0 + C(ses)"}).fit()
    
    # Get predicted values for a range of electrode values
    electrode_values = np.linspace(corr_df_clean_trust[electrode].min(), corr_df_clean_trust[electrode].max(), 100)
    predicted_performance = model.params['Intercept'] + model.params[electrode] * electrode_values
    
    # Determine the subplot to use
    ax = axs[i // 2, i % 2]
    
    # Plot the observed data using the specified color
    sns.scatterplot(x=electrode, y='cognitive_trust', data=corr_df_clean_trust, color='#8c96c6', alpha=0.6, ax=ax)
    
    # Plot the fitted line using the specified color
    ax.plot(electrode_values, predicted_performance, color='#4d004b', label='Fitted Line')
    
    # Set titles and labels
    ax.set_xlabel(electrode)
    ax.set_ylabel('Pairwise Average')
    ax.set_title(f'Mixed Model Fit: Pairwise \nAverage Cognitive Trust ~ {electrode}')
    ax.legend()

# Ensure the background of the figure is white
fig.patch.set_facecolor('white')

plt.tight_layout()
plt.savefig('plots/model_fits_average_cognitive_trust.png', format='png', dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')  # Save the figure
plt.show()


In [ ]:
# Now fit the same model for average affective trust

# Fit a complex mixed model that deals with sessions nested within sujects
# This model:
    # "Cognitive Trust ~ Electrode" defines "Cognitive Trust" as the DV and"AF7" as an independent variable (fixed effect)
    # groups=corr_df["subj"] specifies that the random intercepts should vary by "subj" to account for the non-independence of observations within subjects
    #re_formula="1" indicates that there is a random intercept for each subject
    # vc_formula={"ses": "0 + C(ses)"} specifies a variance component for "ses", treating sessions as a categorical variable (with C(ses)) and allowing for varying intercepts by "ses" without a slope
# This may not converge, so we fit simpler models below

# Define the models
model_af7 = smf.mixedlm("affective_trust ~ AF7", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_af8 = smf.mixedlm("affective_trust ~ AF8", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp9 = smf.mixedlm("affective_trust ~ TP9", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp10 = smf.mixedlm("affective_trust ~ TP10", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

# Fit the model
results_af7 = model_af7.fit()
results_af8 = model_af8.fit()
results_tp9 = model_tp9.fit()
results_tp10 = model_tp10.fit()

# Print the summary of the model fit
print(results_af7.summary())
print(results_af8.summary())
print(results_tp9.summary())
print(results_tp10.summary())

In [ ]:
# Plot the mixed model of electrode on average affective_trust

# Prepare the grid for plotting
fig, axs = plt.subplots(2, 2, figsize=(8.5, 8.5), dpi=300)  # Adjust figsize to 8.5 x 8.5 inches

electrodes = ['AF7', 'AF8', 'TP9', 'TP10']

for i, electrode in enumerate(electrodes):
    # Fit the model for the current electrode
    model = smf.mixedlm(f"affective_trust ~ {electrode}", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"], re_formula="1",
                        vc_formula={"ses": "0 + C(ses)"}).fit()
    
    # Get predicted values for a range of electrode values
    electrode_values = np.linspace(corr_df_clean_trust[electrode].min(), corr_df_clean_trust[electrode].max(), 100)
    predicted_performance = model.params['Intercept'] + model.params[electrode] * electrode_values
    
    # Determine the subplot to use
    ax = axs[i // 2, i % 2]
    
    # Plot the observed data using the specified color
    sns.scatterplot(x=electrode, y='affective_trust', data=corr_df_clean_trust, color='#8c96c6', alpha=0.6, ax=ax)
    
    # Plot the fitted line using the specified color
    ax.plot(electrode_values, predicted_performance, color='#4d004b', label='Fitted Line')
    
    # Set titles and labels
    ax.set_xlabel(electrode)
    ax.set_ylabel('Pairwise Average')
    ax.set_title(f'Mixed Model Fit: Pairwise \nAverage Affective Trust ~ {electrode}')
    ax.legend()

# Ensure the background of the figure is white
fig.patch.set_facecolor('white')

plt.tight_layout()
plt.savefig('plots/model_fits_average_affective_trust.png', format='png', dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')  # Save the figure
plt.show()

In [ ]:
# Now fit the same model for pairwise cognitive trust correlations

# Fit a complex mixed model that deals with sessions nested within sujects
# This model:
    # "Cognitive Trust ~ Electrode" defines "Cognitive Trust" as the DV and"AF7" as an independent variable (fixed effect)
    # groups=corr_df["subj"] specifies that the random intercepts should vary by "subj" to account for the non-independence of observations within subjects
    #re_formula="1" indicates that there is a random intercept for each subject
    # vc_formula={"ses": "0 + C(ses)"} specifies a variance component for "ses", treating sessions as a categorical variable (with C(ses)) and allowing for varying intercepts by "ses" without a slope
# This may not converge, so we fit simpler models below

# Define the models
model_af7 = smf.mixedlm("cognitivetrust_corr ~ AF7", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_af8 = smf.mixedlm("cognitivetrust_corr ~ AF8", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp9 = smf.mixedlm("cognitivetrust_corr ~ TP9", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp10 = smf.mixedlm("cognitivetrust_corr ~ TP10", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

# Fit the model
results_af7 = model_af7.fit()
results_af8 = model_af8.fit()
results_tp9 = model_tp9.fit()
results_tp10 = model_tp10.fit()

# Print the summary of the model fit
print(results_af7.summary())
print(results_af8.summary())
print(results_tp9.summary())
print(results_tp10.summary())

In [ ]:
# Plot the mixed model of electrode on pairwise cognitive_trust correlation

# Prepare the grid for plotting
fig, axs = plt.subplots(2, 2, figsize=(8.5, 8.5), dpi=300)  # Adjust figsize to 8.5 x 8.5 inches

electrodes = ['AF7', 'AF8', 'TP9', 'TP10']

for i, electrode in enumerate(electrodes):
    # Fit the model for the current electrode
    model = smf.mixedlm(f"cognitivetrust_corr ~ {electrode}", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"], re_formula="1",
                        vc_formula={"ses": "0 + C(ses)"}).fit()
    
    # Get predicted values for a range of electrode values
    electrode_values = np.linspace(corr_df_clean_trust[electrode].min(), corr_df_clean_trust[electrode].max(), 100)
    predicted_performance = model.params['Intercept'] + model.params[electrode] * electrode_values
    
    # Determine the subplot to use
    ax = axs[i // 2, i % 2]
    
    # Plot the observed data using the specified color
    sns.scatterplot(x=electrode, y='cognitivetrust_corr', data=corr_df_clean_trust, color='#8c96c6', alpha=0.6, ax=ax)
    
    # Plot the fitted line using the specified color
    ax.plot(electrode_values, predicted_performance, color='#4d004b', label='Fitted Line')
    
    # Set titles and labels
    ax.set_xlabel(electrode)
    ax.set_ylabel('Pairwise Correlation')
    ax.set_title(f'Mixed Model Fit: Pairwise \nCognitive Trust Correlation ~ {electrode}')
    ax.legend()

# Ensure the background of the figure is white
fig.patch.set_facecolor('white')

plt.tight_layout()
plt.savefig('plots/model_fits_pairwise_cognitive_trust_correlation.png', format='png', dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')  # Save the figure
plt.show()


In [ ]:
# NOTE: AF8 MODEL IS SINGULAR!!!

# Now fit the same model for pairwise affective trust correlations

# Fit a complex mixed model that deals with sessions nested within sujects
# This model:
    # "Cognitive Trust ~ Electrode" defines "Cognitive Trust" as the DV and"AF7" as an independent variable (fixed effect)
    # groups=corr_df["subj"] specifies that the random intercepts should vary by "subj" to account for the non-independence of observations within subjects
    #re_formula="1" indicates that there is a random intercept for each subject
    # vc_formula={"ses": "0 + C(ses)"} specifies a variance component for "ses", treating sessions as a categorical variable (with C(ses)) and allowing for varying intercepts by "ses" without a slope
# This may not converge, so we fit simpler models below

# Define the models
model_af7 = smf.mixedlm("affecttrust_corr ~ AF7", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_af8 = smf.mixedlm("affecttrust_corr ~ AF8", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp9 = smf.mixedlm("affecttrust_corr ~ TP9", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

model_tp10 = smf.mixedlm("affecttrust_corr ~ TP10", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"],
                    re_formula="1", vc_formula={"ses": "0 + C(ses)"})

# Fit the model
results_af7 = model_af7.fit()
results_af8 = model_af8.fit()
results_tp9 = model_tp9.fit()
results_tp10 = model_tp10.fit()

# Print the summary of the model fit
print(results_af7.summary())
print(results_af8.summary())
print(results_tp9.summary())
print(results_tp10.summary())

In [ ]:
# Plot the mixed model of electrode on pairwise affective_trust correlation

# Prepare the grid for plotting
fig, axs = plt.subplots(2, 2, figsize=(8.5, 8.5), dpi=300)  # Adjust figsize to 8.5 x 8.5 inches

electrodes = ['AF7', 'AF8', 'TP9', 'TP10']

for i, electrode in enumerate(electrodes):
    # Fit the model for the current electrode
    model = smf.mixedlm(f"affecttrust_corr ~ {electrode}", data=corr_df_clean_trust, groups=corr_df_clean_trust["subj"], re_formula="1",
                        vc_formula={"ses": "0 + C(ses)"}).fit()
    
    # Get predicted values for a range of electrode values
    electrode_values = np.linspace(corr_df_clean_trust[electrode].min(), corr_df_clean_trust[electrode].max(), 100)
    predicted_performance = model.params['Intercept'] + model.params[electrode] * electrode_values
    
    # Determine the subplot to use
    ax = axs[i // 2, i % 2]
    
    # Plot the observed data using the specified color
    sns.scatterplot(x=electrode, y='affecttrust_corr', data=corr_df_clean_trust, color='#8c96c6', alpha=0.6, ax=ax)
    
    # Plot the fitted line using the specified color
    ax.plot(electrode_values, predicted_performance, color='#4d004b', label='Fitted Line')
    
    # Set titles and labels
    ax.set_xlabel(electrode)
    ax.set_ylabel('Pairwise Correlation')
    ax.set_title(f'Mixed Model Fit: Pairwise \nAffective Trust Correlation ~ {electrode}')
    ax.legend()

# Ensure the background of the figure is white
fig.patch.set_facecolor('white')

plt.tight_layout()
plt.savefig('plots/model_fits_pairwise_affective_trust_correlation.png', format='png', dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')  # Save the figure
plt.show()
